In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql import Window
from pyspark.sql import Row, functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as funcs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col

import boto3
import datetime
import re
import numpy as np
import pandas as pd

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1620513438897_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

this version of pandas is incompatible with numpy < 1.15.4
your numpy version is 1.14.5.
Please upgrade numpy to >= 1.15.4 to use this pandas version
Traceback (most recent call last):
  File "/usr/local/lib64/python3.6/site-packages/pandas/__init__.py", line 22, in <module>
    from pandas.compat.numpy import (
  File "/usr/local/lib64/python3.6/site-packages/pandas/compat/numpy/__init__.py", line 21, in <module>
    "this version of pandas is incompatible with numpy < 1.15.4\n"
ImportError: this version of pandas is incompatible with numpy < 1.15.4
your numpy version is 1.14.5.
Please upgrade numpy to >= 1.15.4 to use this pandas version



In [2]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("SparkLogs") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'SparkSession' is not defined
Traceback (most recent call last):
NameError: name 'SparkSession' is not defined



In [3]:
# Load book meta data
df_books_raw = spark.read.json("s3://book-recommender-goodreads/goodreads_books.json.gz")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_books_raw.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- asin: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- format: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- link: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- popular_shelves: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- count: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- pub

In [4]:
# Trim the dataframe to columns of interest
df_books_trim = df_books_raw.select("title", "book_id", "publication_year", "ratings_count", "language_code", "average_rating", "popular_shelves")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def array_to_string(my_list):
    """Convert JSON dict to list of strings"""
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

array_to_string_udf = udf(array_to_string, StringType())

df_books_trim = df_books_trim.withColumn('popular_shelves', array_to_string_udf(df_books_trim["popular_shelves"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Filter for only specific genres using regex
expression = "economics|politics|sociology|society|political-science|government|history|philosophy|theory"
df_books_genres = df_books_trim.filter(df_books_trim['popular_shelves'].rlike(expression))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Save data to S3 bucket
df_books_genres.write.format("csv").option("header","true").mode("Overwrite").save("s3a://book-recommender-goodreads/booksdata-genres")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_books_genres.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

111119

## User Interactions

In [8]:
# Load user interaction data
df_book_inter = spark.read.option("header",True).csv("s3://book-recommender-goodreads/goodreads_interactions.csv")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_book_inter.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- is_read: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- is_reviewed: string (nullable = true)

In [47]:
df_book_inter.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-------+------+-----------+
|user_id|book_id|is_read|rating|is_reviewed|
+-------+-------+-------+------+-----------+
|      0|    948|      1|     5|          0|
|      0|    947|      1|     5|          1|
|      0|    946|      1|     5|          0|
+-------+-------+-------+------+-----------+
only showing top 3 rows

In [9]:
book_id_list = df_books_genres.select('book_id').collect()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
book_id_array = [str(row.book_id) for row in book_id_list]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_select_book_inter = df_book_inter.filter(df_book_inter.book_id.isin(book_id_array))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_select_book_inter = df_select_book_inter.filter(df_book_inter.is_read == 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df_select_book_inter.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20576730

In [69]:
df_select_book_inter_10m = df_select_book_inter.limit(10000000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# Save data to S3 bucket
df_select_book_inter_10m.write.format("csv").option("header","true").mode("Overwrite") \
                              .save("s3a://book-recommender-goodreads/book-inter-10m")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…